In [1]:
import numpy as np
import os
import math
import pickle
from collections import Counter

from sklearn import linear_model,cross_validation
from sklearn.cross_validation import KFold
from sklearn import grid_search

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


In [7]:
# "bestLabelForGroup(labelList)" is used for 
# group wise prediction.
# Return the best label for the group, 
# Returns label that is most likely to be in 
# the group of amino acids.
def bestLabelForGroup(labelList):   
    mostCommonLabel, count_mostCommonLabel = Counter(labelList).most_common(1)[0]
    return mostCommonLabel

def sigmoidfn(x):
    return 1/(1+math.exp(-x))

In [8]:
# Open the test-72 protein list
foldFileName = 'prot_list'
foldFile = open(foldFileName, 'r+')

# Load the classifiers
f=open('l1_logreg.pickle')
l1_logreg = pickle.load(f)
f.close()

f=open('l1_logreg_gridsearch.pickle')
l1_logreg_gridsearch = pickle.load(f)
f.close()

# f=open('sgdc.pickle')
# sgdc = pickle.load(f)
# f.close()


# f=open('rfeET.pickle')
# rfeET = pickle.load(f)
# f.close()


# f=open('dtree.pickle')
# dtree = pickle.load(f)
# f.close()

In [9]:
accuracyArr = []
mccArr = []
fScoreArr = []

for proteinName in foldFile.readlines():
    
    # Flag to account for any processing errors while building the feature vectors
    flag = 0
    
    # Load PRSA file for each protein 
    prsaFileName = './PRSA/'+proteinName[:-1]+'.prsa'
    prsaFileArr = np.loadtxt(prsaFileName, usecols=(3,))
    # Load ss2 file
    ss2FileName = './psipred/'+proteinName[:-1]+'.ss2'
    ss2FileArr = np.loadtxt(ss2FileName, skiprows=2, usecols=(0,3,4,5), ndmin=2)
    # Load pssm file
    pssmFileName = './PSSM/'+proteinName[:-1]+'.pssm'
    pssmFile = open(pssmFileName, 'r+')
    pssmList = []
    
    i = 0
    for lines in pssmFile.readlines():
        if i<3:
            pass
        else:
            try:
                pssmList.append([int(lines[i:i+3]) for i in range(9,len(lines)-94,3)])     #162-94
            except:
                #print proteinName, "Check line ", i
                flag = 1
                break

        i+=1

    # Just accounting for the errors in processing in pssm file
    if flag == 1:
        continue
    else:
        pass
    
    # Load PPI-site File for final comparison after prediction
    # Only ppiFileArr[4] through to ppiFileArr[-5] is used in steps of 5
    # for each protein because negihbouring acids' values are taken per amino-acid-group
    ppiFileName = './PPI-site/'+proteinName[:-1]+'.int'
    ppiFileArr = np.loadtxt(ppiFileName,usecols=(0,))

    yActualArr = []
    # yActualArr = ppiFileArr[4:-5]         #Will be used for final testing (one by one classification)
    for pj in range(4,len(ppiFileArr)-5,5):
        yActualArr.append( bestLabelForGroup([ppiFileArr[x] for x in range(pj-4,pj+6)]) )
        
    X = []      # Set of feature vectors to be fed into the classifer

    j = 4       # Start with the 4th position for each protein and move on to the L-5th
                # j is the position of center of each amino-acid-group (10 per group) within the protein

    while j<len(ppiFileArr)-5:
        # 1. Get 10 PSSM value sets - each is a set of 20 values => 10*20 = 200
        
        x = []  # Per-amino-acid feature vector

        i = j-4
        while i<j+6:
            try:
                for n in pssmList[i]:
                    x.append(sigmoidfn(n))
            except:
                print "some error while getting pssm values"
            i+=1

        # 2. Get 10 ss2 value-sets (each is a set of 3 values)      => 30 values
        i = j-4
        while i<j+6:
            x.append(ss2FileArr[i][1])
            x.append(ss2FileArr[i][2])
            x.append(ss2FileArr[i][3])

            i+=1 

        # 3. get 10 prsa values
        i = j-4
        while i<j+6:
            x.append(prsaFileArr[i])
            i+=1

        X.append(x)         # Readying each feature vector
        
        j+=5  # Increase j in steps of 5 to move to the next group
   

    del pssmList[:]
    np.delete(ss2FileArr, [i for i in range(len(ss2FileArr))])
    np.delete(prsaFileArr, [i for i in range(len(prsaFileArr))])
    np.delete(ppiFileArr, [i for i in range(len(ppiFileArr))])
    pssmFile.close()
    
    # i11 += 1

    # print len(X), len(X[0])
    # print len(yActualArr)

    yPredArr = []
    yPredArr = l1_logreg_gridsearch.predict(X)
    # print len(yPredArr)
    print proteinName, "- pred using l1_logreg (with grid search):"
    accuracyArr.append(l1_logreg.score(X,yActualArr) )
    mccArr.append(matthews_corrcoef(yActualArr,yPredArr))
    fScoreArr.append(f1_score(yActualArr,yPredArr))
    print "Accuracy:",accuracyArr[-1]
    print "\tMCC: ",mccArr[-1]
    print "\tF-measure: ",fScoreArr[-1]
    print "CM :"
    print confusion_matrix(yActualArr,yPredArr,labels=[1, -1]) # Predicted on top (i.e., vertial)
    print

    # yPredArr = sgdc.predict(X)
    # print proteinName, "- pred using sgdc:"
    # accuracyArr.append(sgdc.score(X,yActualArr) )
    # mccArr.append(matthews_corrcoef(yActualArr,yPredArr))
    # fScoreArr.append(f1_score(yActualArr,yPredArr))
    # print "Accuracy:",accuracyArr[-1]
    # print "\tMCC: ",mccArr[-1]
    # print "\tF-measure: ",fScoreArr[-1]
    # print "CM :"
    # print confusion_matrix(yActualArr,yPredArr,labels=[1, -1])
    # print

    # yPredArr = rfeET.predict(X)
    # print proteinName, "- pred using rfeET:"
    # accuracyArr.append(rfeET.score(X,yActualArr) )
    # mccArr.append(matthews_corrcoef(yActualArr,yPredArr))
    # fScoreArr.append(f1_score(yActualArr,yPredArr))
    # print "Accuracy:",accuracyArr[-1]
    # print "\tMCC: ",mccArr[-1]
    # print "\tF-measure: ",fScoreArr[-1]
    # print "CM :"
    # print confusion_matrix(yActualArr,yPredArr,labels=[1, -1])
    # print

    # yPredArr = dtree.predict(X)
    # print proteinName, "- pred using dtree:"
    # accuracyArr.append(dtree.score(X,yActualArr) )
    # mccArr.append(matthews_corrcoef(yActualArr,yPredArr))
    # fScoreArr.append(f1_score(yActualArr,yPredArr))
    # print "Accuracy:",accuracyArr[-1]
    # print "\tMCC: ",mccArr[-1]
    # print "\tF-measure: ",fScoreArr[-1]
    # print "CM :"
    # print confusion_matrix(yActualArr,yPredArr,labels=[1, -1])
    # print

    del X[:]
    # np.delete(yPredArr, [i for i in range(len(yPredArr))])
    np.delete(yActualArr, [i for i in range(len(yActualArr))])

print "Total Average Accuracy for test-72: ", float(sum(accuracyArr))/len(accuracyArr)
print "Total Average MCC Score for test-72: ", float(sum(mccArr))/len(mccArr)
print "Total Average f-score for test 72: ", float(sum(fScoreArr))/len(fScoreArr)
print 
print "Max Accuracy obtained for a chain in test-72: ", max(accuracyArr)
print "Max MCC obtained for a chain in test-72: ", max(mccArr)
print "Max f-score obtained for a chain in test-72: ", max(fScoreArr)
print 
print "Min Accuracy obtained for a chain in test-72: ", min(accuracyArr)
print "Min MCC obtained for a chain in test-72: ", min(mccArr)
print "Min f-score obtained for a chain in test-72: ", min(fScoreArr)    

posmcc = 0
posmccSum = 0
negmcc = 0
zeromcc = 0
for i in range(len(mccArr)):
    if mccArr[i]>0:
        posmcc+=1
        posmccSum+=mccArr[i]
    elif mccArr[i]<0:
        negmcc+=1
    else:
        zeromcc+=1

print
print "Number of positive mcc values: ", posmcc
print "Avg of only pos mcc values: ", float(posmccSum)/posmcc
print "Number of negative mcc values: ", negmcc
print "Number of zero mcc values: ", zeromcc
    # Remove
    # if i11>5:   
    #     break

    
"""
Final Comments:

Unable to process the PSSM file for 2 protein chains out of the 60 proteins 1pxv_A and 1pxv_C.


A high number of negative examples in test set is giving - bad results


"""

1ak4_A
- pred using l1_logreg (with grid search):
Accuracy: 1.0
	MCC:  0.0
	F-measure:  0.0
CM :
[[ 0  0]
 [ 0 32]]

1ak4_D
- pred using l1_logreg (with grid search):
Accuracy: 0.928571428571
	MCC:  -0.0533760512684
	F-measure:  0.0
CM :
[[ 0  2]
 [ 1 25]]

1b6c_A
- pred using l1_logreg (with grid search):
Accuracy: 0.75
	MCC:  0.350438322025
	F-measure:  0.285714285714
CM :
[[ 1  5]
 [ 0 14]]

1b6c_B
- pred using l1_logreg (with grid search):
Accuracy: 0.921875
	MCC:  -0.0398304178903
	F-measure:  0.0
CM :
[[ 0  3]
 [ 2 59]]

1dfj_E
- pred using l1_logreg (with grid search):
Accuracy: 0.782608695652
	MCC:  0.0
	F-measure:  0.0
CM :
[[ 0  5]
 [ 0 18]]

1dfj_I
- pred using l1_logreg (with grid search):
Accuracy: 0.955555555556
	MCC:  0.0
	F-measure:  0.0
CM :
[[ 0  4]
 [ 0 86]]

1e4k_C
- pred using l1_logreg (with grid search):
Accuracy: 0.909090909091
	MCC:  0.0
	F-measure:  0.0
CM :
[[ 0  3]
 [ 0 30]]

1e6e_A
- pred using l1_logreg (with grid search):
Accuracy: 0.944444444444
	MCC:  0

'\nFinal Comments:\n\nUnable to process the PSSM file for 2 protein chains out of the 60 proteins 1pxv_A and 1pxv_C.\n\n\nA high number of negative examples in test set is giving - bad results\n\n\n'